In [ ]:
import pandas as pd
import sqlite3

# Load the dataset
df = pd.read_csv("data/RealEstate_California.csv")

# Drop rows where hasBadGeocode is 1 (true)
df = df[df["hasBadGeocode"] != 1]

# Drop unneeded columns
df = df.drop(columns=['stateId', 'cityId', 'countyId', 'is_bankOwned', 'description', 'hasBadGeocode', 'is_forAuction', 'country', 
            'currency', 'state', 'id', 'Unnamed: 0', 'time', 'levels', 'livingAreaValue', 
            'lotAreaUnits', 'buildingArea', 'garageSpaces'])


In [3]:
# Connect to SQLite and create real_estate database
conn = sqlite3.connect("real_estate.db")

# Upload DataFrame to SQLite
df.to_sql('real_estate', conn, if_exists='replace', index=False)

# Test query to grab first 5 from the database
query = "SELECT * FROM real_estate LIMIT 5"
df_test = pd.read_sql_query(query, conn)

conn.close()

df_test.head()


,datePostedString,event,price,pricePerSquareFoot,city,yearBuilt,streetAddress,zipcode,longitude,latitude,...,bathrooms,bedrooms,parking,hasGarage,pool,spa,isNewConstruction,hasPetsAllowed,homeType,county
0,2021-01-13,Listed for sale,145000.0,0.0,Gold Run,0,0 Moody Ridge Rd,95717.0,-120.834473,39.167866,...,0.0,0.0,0,0,0,0,0,0,LOT,Placer County
1,2021-07-12,Listed for sale,675000.0,404.0,Pinole,1958,1476 Belden Ct,94564.0,-122.300613,38.001213,...,2.0,3.0,1,1,0,0,0,0,SINGLE_FAMILY,Contra Costa County
2,2021-07-08,Listed for sale,649000.0,459.0,Pinole,1959,3540 Savage Ave,94564.0,-122.271362,37.982040,...,2.0,3.0,1,1,0,0,0,0,SINGLE_FAMILY,Contra Costa County
3,2021-07-07,Listed for sale,599000.0,448.0,Pinole,1908,2391 Plum St,94564.0,-122.289230,38.004395,...,2.0,3.0,1,1,0,1,0,0,SINGLE_FAMILY,Contra Costa County
4,2021-07-07,Listed for sale,299000.0,0.0,Pinole,0,2693 Appian Way,94564.0,-122.298439,37.986309,...,0.0,0.0,0,0,0,0,0,0,LOT,Contra Costa County
